In [12]:
from pyardrone import ARDrone, at
from contextlib import suppress
from threading import Timer
import math
import time
import numpy as np
from zmq import Message
import cv2
import os
from time import sleep, perf_counter
import matplotlib
matplotlib.use('TkAgg')  # Pastikan menggunakan backend GUI yang tepat
import matplotlib.pyplot as plt


In [20]:
drone = ARDrone(host="192.168.1.1")


In [ ]:
drone.send(at.CONFIG('general:navdata_demo', True))

In [ ]:

drone.navdata

In [1]:
from pyardrone import ARDrone, at
from time import sleep, perf_counter
import numpy as np

drone = None
battery = None
vx = None
vy = None
vz = None
yaw = None
pitch = None
roll = None
altitude = None
flying_state = None
navdemo = None
camera_condition = "2"
sending_state = False
current_time = 0

setup_moving = False
running_program = True
moving_program = False
direction = 0
kec = 0.2

id = 0
ip = '192.168.1.1'


# Fungsi untuk menyimpan Navdata ke dalam DataFrame
navdata_records = [['current_time , battery , flying_state , altitude , vx , vy , vz , yaw , pitch , roll']]

def connect(drone, ip, navdemo):
    if drone is None:
        try:
            drone = ARDrone(host=ip)
            drone.send(at.CONFIG('general:navdata_demo', True))
            connected = True
        except:
            print("can not connected")
            drone = None
            connected = False
    else:
        if drone.state.navdata_bootstrap is not True:
            navdemo = drone.navdata.demo
            print("connected")
            connected = True
        else:
            print("navdata demo is problem")
            drone.send(at.CONFIG('general:navdata_demo', True))
            sleep(1.5)
            navdemo = drone.navdata.demo
            connected = False
    sleep(0.5)
    return drone, navdemo, connected

def disconnect(drone):
    drone.close()
    drone = None
    return drone

def get_battery(drone,navdemo):
    if drone is not None:
        if drone.navdata_ready.is_set():
            if navdemo is not None:
                battery = navdemo.vbat_flying_percentage
                print(battery)
    else:
        battery = "Not connected"
    return battery

def get_navdata(drone,navdemo,sending_state,current_time,navdata_records):
    if sending_state is False:
        if drone is not None:
            if navdemo is not None:
                if drone.state.navdata_bootstrap is not True:
                    navdemo = drone.navdata.demo
                    vx = navdemo.vx/1000
                    vy = navdemo.vy/1000
                    vz = navdemo.vz/1000
                    yaw = navdemo.psi/1000
                    pitch = navdemo.theta/1000
                    roll = navdemo.phi/1000
                    altitude = navdemo.altitude/1000
                    flying_state = drone.state.fly_mask
                    battery = navdemo.vbat_flying_percentage
                    navdata_records.append([current_time , battery , flying_state , altitude , vx , vy , vz , yaw , pitch , roll])
            else:
                if drone.state.navdata_bootstrap is not True:
                    navdemo = drone.navdata.demo
                else:
                    print("navdata demo is problem")
                    drone.send(at.CONFIG('general:navdata_demo', True))
    return navdemo, navdata_records
    

def move(drone, forward=0.0, backward=0.0, right=0.0, left=0.0, up=0.0, down=0.0, cw=0.0, ccw=0.0, dir_speed=0.0):
    if drone != None:
        drone.move(forward=forward, backward=backward,right=right, left=left, up=up, down=down, cw=cw, ccw=ccw)
    elif drone == None:
        print("Drone belum terhubung")

def hover(drone):
    if drone != None:
        drone.move(forward=0.0, backward=0.0, right=0.0, left=0.0, up=0.0, down=0.0, cw=0.0, ccw=0.0)
        print('drone berhasil hover')
        sleep(1)
    elif drone == None:
        print("Drone belum terhubung")

def takeoff(drone):
    print('takeoff')
    drone.takeoff()

def land(drone):
    print('landing')
    drone.land()

def read_config():
    pass

def trim(drone):
    drone.send(at.FTRIM())
    sleep(2)
    # drone.send(
    #     at.CONFIG('video:bitrate_control_mode', '1'))
    # sleep(1)
    # drone.send(
    #     at.CONFIG('video:video_fps', '30'))
    # sleep(1)
    # drone.send(
    #     at.CONFIG('video:video_codec', '129'))
    # sleep(1)
    # drone.send(
    #     at.CONFIG('video:max_bitrate', '4000'))

def calib(drone):
    drone.send(at.CALIB(0))

def change_camera2(drone):
    sending_state = True
    camera_condition = "2"
    # sleep(0.5)
    drone.send(at.CONFIG("video:video_channel", camera_condition))
    sending_state = False
    return camera_condition

def change_camera3(drone):
    sending_state = True
    camera_condition = "3"
    # sleep(0.5)
    drone.send(at.CONFIG("video:video_channel", camera_condition))
    sending_state = False
    return camera_condition

In [2]:

waktu_input = 0.1
waktu_move = 3
drone, navdemo, connected = connect(drone, ip, navdemo)

try : 
    if drone != None:
    
        init_time = perf_counter()
        last_time_sampling = init_time
        last_time_move = init_time

        drone.navdata_ready.wait()  # wait until NavData is ready
        # trim()
        # print("Drone berhasil trim")

        while running_program == True:
        
            current_time = perf_counter()
            navdemo, navdata_records =  get_navdata(drone,navdemo,sending_state,current_time,navdata_records)
          
            if setup_moving == False:    
                takeoff(drone)
                sleep(5)  # Tunggu beberapa detik untuk memastikan drone stabil
                print("Drone Take off")
                # calib()
                # time.sleep(5)
                # print("Drone berhasil Kalibrasi")
                setup_moving = True

            if (((current_time - last_time_move) >= waktu_move) and (setup_moving == True)) :
                
                if (direction == 0):
                    direction = 1
                    print("Drone Maju")

                elif (direction == 1):
                    hover(drone)
                    direction = 2
                    print("Drone Ke kanan")

                elif (direction == 2):
                    hover(drone)
                    direction = 3
                    print("Drone ke belakang")
                
                elif (direction == 3):
                    hover(drone)
                    direction = 4
                    print("Drone ke kiri")
                
                elif (direction == 4):
                    hover(drone)
                    direction = 5
                    print("Trajektori selesai, akan mendarat")
                
                last_time_move = current_time

            if setup_moving == True :

                if (direction == 1):
                    move(drone, forward=kec, backward=0.0, right=0.0, left=0.0, up=0.0, down=0.0, cw=0.0, ccw=0.0, dir_speed=0.0)

                elif (direction == 2):
                    move(drone, forward=0.0, backward=0.0, right=kec, left=0.0, up=0.0, down=0.0, cw=0.0, ccw=0.0, dir_speed=0.0)
                
                elif (direction == 3):
                    move(drone, forward=0.0, backward=kec, right=0.0, left=0.0, up=0.0, down=0.0, cw=0.0, ccw=0.0, dir_speed=0.0)
                
                elif (direction == 4):
                    move(drone, forward=0.0, backward=0.0, right=0.0, left=kec, up=0.0, down=0.0, cw=0.0, ccw=0.0, dir_speed=0.0)
                
                elif (direction == 5):
                    land(drone)
                    print("Drone Mendarat")
                    running_program == False
                    break

except KeyboardInterrupt:
    print("Program dihentikan oleh pengguna. Drone akan Mendarat")
    land()
    print("Drone mendarat")
    running_program == False

np.savetxt("testDataArray.txt", navdata_records, fmt="%s")
drone = disconnect(drone)

takeoff
Drone Take off
Drone Maju
drone berhasil hover
Drone Ke kanan
drone berhasil hover
Drone ke belakang
drone berhasil hover
Drone ke kiri
drone berhasil hover
Trajektori selesai, akan mendarat
landing
Drone Mendarat


c:\Python310\lib\site-packages\numpy\lib\npyio.py:1513: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X = np.asarray(X)


In [ ]:
from pyardrone import ARDrone, at
from time import sleep, perf_counter
import numpy as np

def connect(drone, ip, navdemo):
    if drone is None:
        try:
            drone = ARDrone(host=ip)
            drone.send(at.CONFIG('general:navdata_demo', True))
            connected = True
        except:
            print("can not connected")
            drone = None
            connected = False
    else:
        if drone.state.navdata_bootstrap is not True:
            navdemo = drone.navdata.demo
            print("connected")
            connected = True
        else:
            print("navdata demo is problem")
            drone.send(at.CONFIG('general:navdata_demo', True))
            sleep(1.5)
            navdemo = drone.navdata.demo
            connected = False
    sleep(0.5)
    return drone, navdemo, connected

def disconnect(drone):
    drone.close()
    drone = None
    return drone

def get_battery(drone,navdemo):
    if drone is not None:
        if drone.navdata_ready.is_set():
            if navdemo is not None:
                battery = navdemo.vbat_flying_percentage
                print(battery)
    else:
        battery = "Not connected"
    return battery

def get_navdata(drone,navdemo,sending_state,current_time,navdata_records):
    if sending_state is False:
        if drone is not None:
            if navdemo is not None:
                if drone.state.navdata_bootstrap is not True:
                    navdemo = drone.navdata.demo
                    vx = navdemo.vx/1000
                    vy = navdemo.vy/1000
                    vz = navdemo.vz/1000
                    yaw = navdemo.psi/1000
                    pitch = navdemo.theta/1000
                    roll = navdemo.phi/1000
                    altitude = navdemo.altitude/1000
                    flying_state = drone.state.fly_mask
                    battery = navdemo.vbat_flying_percentage
                    navdata_records.append([current_time , battery , flying_state , altitude , vx , vy , vz , yaw , pitch , roll])
            else:
                if drone.state.navdata_bootstrap is not True:
                    navdemo = drone.navdata.demo
                else:
                    print("navdata demo is problem")
                    drone.send(at.CONFIG('general:navdata_demo', True))
    return navdemo, navdata_records
    

def move(drone, forward=0.0, backward=0.0, right=0.0, left=0.0, up=0.0, down=0.0, cw=0.0, ccw=0.0, dir_speed=0.0):
    if drone != None:
        drone.move(forward=forward, backward=backward,right=right, left=left, up=up, down=down, cw=cw, ccw=ccw)
    elif drone == None:
        print("Drone belum terhubung")

def hover(drone):
    if drone != None:
        drone.move(forward=0.0, backward=0.0, right=0.0, left=0.0, up=0.0, down=0.0, cw=0.0, ccw=0.0)
        print('drone berhasil hover')
        sleep(1)
    elif drone == None:
        print("Drone belum terhubung")

def takeoff(drone):
    print('takeoff')
    drone.takeoff()

def land(drone):
    print('landing')
    drone.land()

def read_config():
    pass

def trim(drone):
    drone.send(at.FTRIM())
    sleep(2)
    # drone.send(
    #     at.CONFIG('video:bitrate_control_mode', '1'))
    # sleep(1)
    # drone.send(
    #     at.CONFIG('video:video_fps', '30'))
    # sleep(1)
    # drone.send(
    #     at.CONFIG('video:video_codec', '129'))
    # sleep(1)
    # drone.send(
    #     at.CONFIG('video:max_bitrate', '4000'))

def calib(drone):
    drone.send(at.CALIB(0))

def change_camera2(drone):
    sending_state = True
    camera_condition = "2"
    # sleep(0.5)
    drone.send(at.CONFIG("video:video_channel", camera_condition))
    sending_state = False
    return camera_condition

def change_camera3(drone):
    sending_state = True
    camera_condition = "3"
    # sleep(0.5)
    drone.send(at.CONFIG("video:video_channel", camera_condition))
    sending_state = False
    return camera_condition

drone = None
battery = None
vx = None
vy = None
vz = None
yaw = None
pitch = None
roll = None
altitude = None
flying_state = None
navdemo = None
camera_condition = "2"
sending_state = False
current_time = 0

setup_moving = False
running_program = True
moving_program = False
direction = 0
kec = 0.2

id = 0
ip = '192.168.1.1'


# Fungsi untuk menyimpan Navdata ke dalam DataFrame
navdata_records = [['current_time , battery , flying_state , altitude , vx , vy , vz , yaw , pitch , roll']]


waktu_input = 0.1
waktu_move = 3
drone, navdemo, connected = connect(drone, ip, navdemo)

try : 
    if drone != None:
    
        init_time = perf_counter()
        last_time_sampling = init_time
        last_time_move = init_time

        drone.navdata_ready.wait()  # wait until NavData is ready
        # trim()
        # print("Drone berhasil trim")

        while running_program == True:
        
            current_time = perf_counter()
            navdemo, navdata_records =  get_navdata(drone,navdemo,sending_state,current_time,navdata_records)
          
            if setup_moving == False:    
                takeoff(drone)
                sleep(5)  # Tunggu beberapa detik untuk memastikan drone stabil
                print("Drone Take off")
                # calib()
                # time.sleep(5)
                # print("Drone berhasil Kalibrasi")
                setup_moving = True

            if (((current_time - last_time_move) >= waktu_move) and (setup_moving == True)) :
                
                if (direction == 0):
                    direction = 1
                    print("Drone Maju")

                elif (direction == 1):
                    hover(drone)
                    direction = 2
                    print("Drone Ke kanan")

                elif (direction == 2):
                    hover(drone)
                    direction = 3
                    print("Drone ke belakang")
                
                elif (direction == 3):
                    hover(drone)
                    direction = 4
                    print("Drone ke kiri")
                
                elif (direction == 4):
                    hover(drone)
                    direction = 5
                    print("Trajektori selesai, akan mendarat")
                
                last_time_move = current_time

            if setup_moving == True :

                if (direction == 1):
                    move(drone, forward=kec, backward=0.0, right=0.0, left=0.0, up=0.0, down=0.0, cw=0.0, ccw=0.0, dir_speed=0.0)

                elif (direction == 2):
                    move(drone, forward=0.0, backward=0.0, right=kec, left=0.0, up=0.0, down=0.0, cw=0.0, ccw=0.0, dir_speed=0.0)
                
                elif (direction == 3):
                    move(drone, forward=0.0, backward=kec, right=0.0, left=0.0, up=0.0, down=0.0, cw=0.0, ccw=0.0, dir_speed=0.0)
                
                elif (direction == 4):
                    move(drone, forward=0.0, backward=0.0, right=0.0, left=kec, up=0.0, down=0.0, cw=0.0, ccw=0.0, dir_speed=0.0)
                
                elif (direction == 5):
                    land(drone)
                    print("Drone Mendarat")
                    running_program == False
                    break

except KeyboardInterrupt:
    print("Program dihentikan oleh pengguna. Drone akan Mendarat")
    land()
    print("Drone mendarat")
    running_program == False

np.savetxt("testDataArray.txt", navdata_records, fmt="%s")
drone = disconnect(drone)

In [7]:
import time
from pyardrone import ARDrone
drone = ARDrone()
drone.navdata_ready.wait()  # wait until NavData is ready
drone.takeoff()


In [2]:
import cv2
print( cv2.getBuildInformation())


General configuration for OpenCV 4.9.0 =====================================
  Version control:               4.9.0

  Extra modules:
    Location (extra):            D:/a/opencv-python/opencv-python/opencv_contrib/modules
    Version control (extra):     4.9.0

  Platform:
    Timestamp:                   2023-12-31T11:22:07Z
    Host:                        Windows 10.0.17763 AMD64
    CMake:                       3.24.2
    CMake generator:             Visual Studio 14 2015
    CMake build tool:            MSBuild.exe
    MSVC:                        1900
    Configuration:               Debug Release

  CPU/HW features:
    Baseline:                    SSE SSE2 SSE3
      requested:                 SSE3
    Dispatched code generation:  SSE4_1 SSE4_2 FP16 AVX AVX2
      requested:                 SSE4_1 SSE4_2 AVX FP16 AVX2 AVX512_SKX
      SSE4_1 (16 files):         + SSSE3 SSE4_1
      SSE4_2 (1 files):          + SSSE3 SSE4_1 POPCNT SSE4_2
      FP16 (0 files):            + SSSE

In [2]:
"""
Created on Sun Apr 06 2019

@author: Chayatorn Supataragul
"""
# OpenCV Python script to buffer the video stream

# import libraries of python OpenCV as cv2
import cv2
import sys
import threading
import queue

# == Start Initialization ==
frame_buffer_size = 2 # The queue size for keeping video frame for processing. Cannot less than 2

# Capture frames from the ip camera
fn = 'tcp://192.168.1.1:5555'

# Set capture device from fn
cap = cv2.VideoCapture(fn)

# The queue for keeping video frame for processing
frame_buffer = queue.Queue(maxsize=frame_buffer_size)

# == End Initialization ==

# De-allocate any associated memory usage and exit the program
def deallocateAndExit():
    # De-allocate any associated memory usage
    cap.release()# release camera
    cv2.destroyAllWindows()# release screen
    sys.exit() # exit program

# This is a thread function to keep reading frames and put the frames into frame_buffer for preventing lag of frames reading.
def rtsp_read_buffer():
    # ret will be False when cap.read() timeout or error
    ret = True
    while (ret):
        # If frame_buffer queue is full, get the first queue element out of the queue
        if frame_buffer.full():
            frame_buffer.get()
        # Read frame-by-frame
        # capturing each frame
        ret, buffer_frame = cap.read()
        # Put the capturing frame to the queue
        frame_buffer.put(buffer_frame)
    # Exit program
    deallocateAndExit()


# Main function to start the program
def main():

    # Start thead functions to continue their task parallelly
    threading.Thread(target=rtsp_read_buffer, daemon=True).start()
    
    # Check cv2.VideoCapture(fn) is open
    while (cap.isOpened()):

        # Check if frame_buffer queue has frames waiting to process or not
        # If some processes are waiting, let calculate it
        if frame_buffer.empty() != True:

            # Get a frame from the frame_buffer queue
            frame = frame_buffer.get()

            frame_out = frame.copy()# output frame

            # Display video
            cv2.imshow('Frame out', frame_out)# display each frame

            # Terminate program
            if cv2.waitKey(1) == ord('q'):# if user click key 'q', program will be closed
                break
    # End while loop

    # Exit program
    deallocateAndExit()

# start process
if __name__ == '__main__':
    main()

Exception in thread Thread-9 (rtsp_read_buffer):
Traceback (most recent call last):
  File "c:\Python310\lib\threading.py", line 1009, in _bootstrap_inner
    self.run()
  File "c:\Python310\lib\threading.py", line 946, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\ACER\AppData\Local\Temp\ipykernel_22812\3825752981.py", line 45, in rtsp_read_buffer
cv2.error: Unknown C++ exception from OpenCV code


SystemExit: 

In [2]:
drone, navdemo, connected = connect(drone, ip, navdemo)

In [176]:
for i in range(1000):
    get_navdata()
np.savetxt("testDataArray.txt", navdata_records, fmt="%s")

In [4]:
trim()

In [6]:
takeoff()

takeoff


In [7]:
calib()

In [1]:
drone = land(drone)

NameError: name 'land' is not defined

In [4]:
drone = disconnect(drone)

In [3]:
camera_condition = change_camera3(drone)

Aruco.py

In [ ]:
import cv2


active_camera = False
active_detector = False
ids = None
draw = False
brightness = 1
contrast = 0.75
ret = False
frame = None
image_scale = 1
id_0 = []

# Inisialisasi dictionary ArUco dan parameter detektor
aruco_dict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_6X6_1000)
aruco_params = cv2.aruco.DetectorParameters()

def connect_camera():
    global ip
    url = ip
    camera_url = 'tcp://'+url+':5555' if url != 0 else 0
    camera = cv2.VideoCapture(camera_url)
    # Periksa apakah kamera berhasil dibuka
    if not camera.isOpened():
        active_camera = False
        print("Error: Tidak dapat membuka kamera.")
    else :
        active_camera = True

def read():
    # Baca frame dari kamera
    ret, frame = camera.read()

    # Periksa apakah frame berhasil dibaca
    if ret == False:
        print("Error: Tidak dapat membaca frame dari kamera.")

def findArucoMarkers():
    # Konversi frame ke grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Deteksi marker pada frame
    corners, ids, rejected = cv2.aruco.detectMarkers(gray, aruco_dict, parameters=aruco_params)
    
    # Gambar deteksi marker pada frame
    if ids is not None:
        cv2.aruco.drawDetectedMarkers(frame, corners, ids)

    elif ids

def get_ids():
    if (len(id_0) != len(ids)) or ((len(id_0) == len(ids)) and (id_0[0] != ids[0])):
        print("ArUco markers detected. IDs: ", ids)
        id_0 = ids
        return ids
    pass

def show():
    # Tampilkan frame dengan deteksi marker
    cv2.imshow('Detected ArUco Markers', frame)

def done():
    camera.release()
    cv2.destroyAllWindows()
    print("done")

def run():
    read()
    if active_camera == True:

        if ret:
            frame = cv2.cvtColor(raw_frame, cv2.COLOR_BGR2RGB)

            if active_detector == True:
                if draw:
                    cv2.aruco.drawDetectedMarkers(frame, corners, ids)

    if active_detector == True:
        if ret:
            findArucoMarkers()

Program pemrosesan aruco tanpa live plotting

In [11]:
#--------------------------------------------------------------#
############################ IMPORT ############################
#--------------------------------------------------------------#

import cv2
import numpy as np
import os
from time import sleep, perf_counter
import matplotlib
matplotlib.use('TkAgg')  # Pastikan menggunakan backend GUI yang tepat
import matplotlib.pyplot as plt

#--------------------------------------------------------------#
####################### DEFINISI FUNCTION ######################
#--------------------------------------------------------------#

def movAvg(window,dataMovAvgX,dataMovAvgY,xEst,yEst):
    if len(dataMovAvgX) >= (window-1):
        dataMovAvgX.append(xEst)
        dataMovAvgY.append(yEst)
        xAvg = (sum(dataMovAvgX)/len(dataMovAvgX))
        yAvg = (sum(dataMovAvgY)/len(dataMovAvgY))
        dataMovAvgX.pop(0)
        dataMovAvgY.pop(0)
    elif len(dataMovAvgX) < (window-1):
        dataMovAvgX.append(xEst)
        dataMovAvgY.append(yEst)
        xAvg = xEst
        yAvg = yEst
    return xAvg, yAvg,dataMovAvgX,dataMovAvgY

def saveImage(output_folder,file_counter,frame):
    # Simpan gambar ke file
    filename = os.path.join(output_folder, f'detected_marker_{file_counter:04d}.png')
    cv2.imwrite(filename, frame)
    file_counter += 1
    return file_counter
    # kalo mau pake fungsi, copas aja gini -> file_counter = saveImage(output_folder,file_counter,frame)

def dataAruco(num_rows,num_cols,jarak):
        
    total_markers = num_rows * num_cols
    databasePos = []
    dataID = []
    i = 1
    for row in range(num_cols):
        for col in range(num_rows):
            posX = jarak*row
            posY = jarak*col
            databasePos.append([posX, posY])
            dataID.append(i)
            i += 1
    # print(np.array(databasePos))
    # print(databasePos[dataID.index(500)])
    return dataID, databasePos

def convertPos(image, ids, corners, dataID, databasePos):

    ###### est posisi berdasarkan database posisi real #####
    sumx = 0
    sumy = 0
    for i in range(len(ids)): 
        sumx += databasePos[dataID.index(ids[i])][0]
        sumy += databasePos[dataID.index(ids[i])][1]

    x = sumx/len(ids)
    y = sumy/len(ids)

    ###### titik tengah setiap marker aruco #####
    tengah = []
    
    for i in range(len(ids)):
        xTengah = (int(corners[i][0][0][0])+int(corners[i][0][1][0])+int(corners[i][0][2][0])+int(corners[i][0][3][0]))/4
        yTengah = (int(corners[i][0][0][1])+int(corners[i][0][1][1])+int(corners[i][0][2][1])+int(corners[i][0][3][1]))/4
        tengah.append([xTengah, yTengah])

    ###### titik tengah rata rata (est posisi dalam pixel) #####
    sumtengahx = 0
    sumtengahy = 0
    for i in range(len(ids)):
        sumtengahx += tengah[i][0]
        sumtengahy += tengah[i][1]

    posPixelX = int(sumtengahx/len(ids))
    posPixelY = int(sumtengahy/len(ids))

    ###### Pusat Kamera #####
    height, width = image.shape[:2]
    pusatKameraX = int(width/2)
    pusatKameraY = int(height/2)

    ###### est posisi pusat kamera #####
    aX = 0.123
    aY = 0.131
    rXPixel = pusatKameraX - posPixelX
    rYPixel = pusatKameraY - posPixelY
    rX = rXPixel * aX
    rY = rYPixel * aY
    xEst = x + rX
    yEst = y + rY

    return x,y,posPixelX,posPixelY,xEst,yEst,pusatKameraX,pusatKameraY,tengah

def cameraDisplay(frame, corners, ids, posPixelX, posPixelY, pusatKameraX, pusatKameraY, tengah):
    uniqDisplay = False
    if ids is not None:
        radius = 5
        #Gambar deteksi marker pada frame
        cv2.aruco.drawDetectedMarkers(frame, corners, ids)
        if uniqDisplay == True:

            #Gambar titik pusat setiap marker aruco yang terdeteksi
            for i in range(len(ids)):
                xTengah = tengah[i][0]
                yTengah = tengah[i][1]
                cv2.circle(frame,(int(xTengah),int(yTengah)), int(radius) , (0,0,255), -1)
        
            # Gambar titik estimasi MFP
            cv2.circle(frame,(posPixelX,posPixelY), int(2*radius), (0,255,0), -1)

            # Gambar titik Pusat kamera
            cv2.circle(frame,(pusatKameraX,pusatKameraY), int(2*radius), (255,0,0), -1)

    # Tampilkan frame dengan deteksi marker
    cv2.imshow('Detected ArUco Markers', frame)

    return

#--------------------------------------------------------------#
######################### SET UP ###############################
#--------------------------------------------------------------#

id_min1 = []
ids = []
tengah = []
dataAkhir = [["Waktu", "X_movAvg", "y_movAvg", "Pusat_x", "Pusat_y", "MFP_X", "MFP_Y", "Pixel_pusat_X", "Pixel_pusat_Y", "Pixel_MFP_X", "Pixel_MFP_Y"]]
num_rows = 25
num_cols = 20
id_1 = []
window = 20
dataMovAvgX = []
dataMovAvgY = []
plotX = []
plotY = []

# Buat folder baru jika belum ada
output_folder = 'detected_frames'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# membuat database posisi tiap marker aruco
dataID, databasePos = dataAruco(num_rows,num_rows,19.5)

# Inisialisasi counter untuk penamaan file berurutan
file_counter = 0

# Inisialisasi waktu
init_time = perf_counter()
current_time = 0

# Inisialisasi kamera
ardrone_url = 'tcp://192.168.1.1:5555'
cap = cv2.VideoCapture(ardrone_url)

# Periksa apakah kamera berhasil dibuka
if not cap.isOpened():
    print("Error: Tidak dapat membuka kamera.")
    exit()

# Inisialisasi dictionary ArUco dan parameter detektor
aruco_dict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_6X6_1000)
aruco_params = cv2.aruco.DetectorParameters()



#--------------------------------------------------------------#
####################### LOOP UTAMA #############################
#--------------------------------------------------------------#

while True:
    current_time = perf_counter()
    # Baca frame dari kamera
    ret, frame = cap.read()

    # Periksa apakah frame berhasil dibaca
    if not ret:
        print("Error: Tidak dapat membaca frame dari kamera.")
        break

    # Konversi frame ke grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Deteksi marker pada frameco_dict, parameters=aruco_params)
    corners, ids, rejected = cv2.aruco.detectMarkers(gray, aruco_dict, parameters=aruco_params)

    # pemrosesan bila terdeteksi adanya marker aruco
    if ids is not None:

        # Convert bacaan MFP menjadi posisi
        x,y,posPixelX,posPixelY,xEst,yEst,pusatKameraX,pusatKameraY,tengah = convertPos(frame, ids, corners, dataID, databasePos)
        xAvg, yAvg, dataMovAvgX, dataMovAvgY = movAvg(window,dataMovAvgX,dataMovAvgY,xEst,yEst)
        # update data x dan y untuk live plotting data
        plotX.append(xAvg)
        plotY.append(yAvg)

        # if plotDisplay == True:
        #     # Register the value of x and y for live plotting data
        #     datax = np.array(plotX)
        #     datay = np.array(plotY)
        #     line1.set_xdata(datax )
        #     line1.set_ydata(datay)

        #save data
        dataAkhir.append([current_time, xAvg, yAvg, xEst, yEst, x, y, pusatKameraX, pusatKameraY, posPixelX, posPixelY])
        
        if (len(id_1) != len(ids)) or ((len(id_1) == len(ids)) and (id_1[0] != ids[0])):
            # print("ArUco markers detected. IDs: ", ids)
            id_1 = ids
    
    cameraDisplay(frame, corners, ids, posPixelX, posPixelY, pusatKameraX, pusatKameraY, tengah)

    
    
    # Tekan 'q' untuk keluar dari loop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        
        break

#--------------------------------------------------------------#
######################### ENDING ###############################
#--------------------------------------------------------------#

# Lepaskan kamera dan tutup jendela
cap.release()
cv2.destroyAllWindows()
np.savetxt("Data_Akhir.txt", dataAkhir, fmt="%s")

plotDisplay = True

if plotDisplay == True:
    # Enable interactive mode
    plt.ion()

    # Create subplot and figure
    figure, ax = plt.subplots(figsize=(10, 8))
    line1, = ax.plot(plotX, plotY, color='blue', marker='.',linewidth=0.5, markersize=15)

    # Set limits for the x and y axis
    ax.set_xlim(0,400)
    ax.set_ylim(0,500)

    # Set labels
    plt.xlabel("X-axis")
    plt.ylabel("Y-axis")
    plt.title("Updating plot...")
    plt.grid(True)

    # Re-draw the figure
    # ax.relim()
    # ax.autoscale_view()
    # figure.canvas.draw()
    # # print("graph update")
    
    # # To flush the GUI events
    # figure.canvas.flush_events()
    # plt.pause(0.1)
    # sleep(0.1)
    # Turn off interactive mode
    plt.ioff()
    # Keep the plot window open
    plt.show(block=True)

Program pemrosesan aruco dengan Live plotting

In [1]:
import cv2
import numpy as np
import os
from time import perf_counter
import threading
import queue

def movAvg(window, dataMovAvgX, dataMovAvgY, xEst, yEst):
    if len(dataMovAvgX) >= (window - 1):
        dataMovAvgX.append(xEst)
        dataMovAvgY.append(yEst)
        xAvg = (sum(dataMovAvgX) / len(dataMovAvgX))
        yAvg = (sum(dataMovAvgY) / len(dataMovAvgY))
        dataMovAvgX.pop(0)
        dataMovAvgY.pop(0)
    else:
        dataMovAvgX.append(xEst)
        dataMovAvgY.append(yEst)
        xAvg = xEst
        yAvg = yEst
    return xAvg, yAvg, dataMovAvgX, dataMovAvgY

def dataAruco(num_rows, num_cols, jarak):
    total_markers = num_rows * num_cols
    databasePos = []
    dataID = []
    i = 1
    for row in range(num_cols):
        for col in range(num_rows):
            posX = jarak * row
            posY = jarak * col
            databasePos.append([posX, posY])
            dataID.append(i)
            i += 1
    return dataID, databasePos

def convertPos(image, ids, corners, dataID, databasePos):
    sumx = 0
    sumy = 0
    for i in range(len(ids)):
        sumx += databasePos[dataID.index(ids[i])][0]
        sumy += databasePos[dataID.index(ids[i])][1]

    x = sumx / len(ids)
    y = sumy / len(ids)

    tengah = []
    for i in range(len(ids)):
        xTengah = (corners[i][0][0][0] + corners[i][0][1][0] + corners[i][0][2][0] + corners[i][0][3][0]) / 4
        yTengah = (corners[i][0][0][1] + corners[i][0][1][1] + corners[i][0][2][1] + corners[i][0][3][1]) / 4
        tengah.append([xTengah, yTengah])

    sumtengahx = 0
    sumtengahy = 0
    for i in range(len(ids)):
        sumtengahx += tengah[i][0]
        sumtengahy += tengah[i][1]

    posPixelX = int(sumtengahx / len(ids))
    posPixelY = int(sumtengahy / len(ids))

    height, width = image.shape[:2]
    pusatKameraX = int(width / 2)
    pusatKameraY = int(height / 2)

    aX = 0.123
    aY = 0.131
    rXPixel = pusatKameraX - posPixelX
    rYPixel = pusatKameraY - posPixelY
    rX = rXPixel * aX
    rY = rYPixel * aY
    xEst = x + rX
    yEst = y + rY

    return x, y, posPixelX, posPixelY, xEst, yEst, pusatKameraX, pusatKameraY, tengah

def cameraDisplay(frame, corners, ids, posPixelX, posPixelY, pusatKameraX, pusatKameraY, tengah):
    if ids is not None:
        radius = 5
        cv2.aruco.drawDetectedMarkers(frame, corners, ids)
        for i in range(len(ids)):
            xTengah = tengah[i][0]
            yTengah = tengah[i][1]
            cv2.circle(frame, (int(xTengah), int(yTengah)), int(radius), (0, 0, 255), -1)
        cv2.circle(frame, (posPixelX, posPixelY), int(2 * radius), (0, 255, 0), -1)
        cv2.circle(frame, (pusatKameraX, pusatKameraY), int(2 * radius), (255, 0, 0), -1)
    cv2.imshow('Detected ArUco Markers', frame)

def process_aruco(queue, stop_event):
    id_min1 = []
    ids = []
    tengah = []
    num_rows = 25
    num_cols = 20
    id_1 = []
    window = 20
    dataMovAvgX = []
    dataMovAvgY = []
    posPixelX = None
    posPixelY = None
    pusatKameraX = None
    pusatKameraY = None

    output_folder = 'detected_frames'
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    dataID, databasePos = dataAruco(num_rows, num_rows, 19.5)
    file_counter = 0
    init_time = perf_counter()
    current_time = 0
    ardrone_url = 'tcp://192.168.1.1:5555'
    cap = cv2.VideoCapture(ardrone_url)

    if not cap.isOpened():
        print("Error: Tidak dapat membuka kamera.")
        stop_event.set()
        return

    aruco_dict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_6X6_1000)
    aruco_params = cv2.aruco.DetectorParameters()

    while not stop_event.is_set():
        current_time = perf_counter()
        ret, frame = cap.read()
        if not ret:
            print("Error: Tidak dapat membaca frame dari kamera.")
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        corners, ids, rejected = cv2.aruco.detectMarkers(gray, aruco_dict, parameters=aruco_params)

        if ids is not None:
            x, y, posPixelX, posPixelY, xEst, yEst, pusatKameraX, pusatKameraY, tengah = convertPos(frame, ids, corners, dataID, databasePos)
            xAvg, yAvg, dataMovAvgX, dataMovAvgY = movAvg(window, dataMovAvgX, dataMovAvgY, xEst, yEst)
            if not queue.full():
                queue.put((xAvg, yAvg))

            if (len(id_1) != len(ids)) or ((len(id_1) == len(ids)) and (id_1[0] != ids[0])):
                id_1 = ids

        cameraDisplay(frame, corners, ids, posPixelX, posPixelY, pusatKameraX, pusatKameraY, tengah)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            stop_event.set()
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    data_queue = queue.Queue(maxsize=10)
    stop_event = threading.Event()
    aruco_thread = threading.Thread(target=process_aruco, args=(data_queue, stop_event))
    aruco_thread.start()

    import matplotlib
    matplotlib.use('TkAgg')
    import matplotlib.pyplot as plt

    plotX = []
    plotY = []

    plt.ion()
    figure, ax = plt.subplots(figsize=(10, 8))
    line1, = ax.plot(plotX, plotY, color='blue', marker='.', linewidth=0.5, markersize=15)
    ax.set_xlim(0, 400)
    ax.set_ylim(0, 500)
    plt.xlabel("X-axis")
    plt.ylabel("Y-axis")
    plt.title("Updating plot...")
    plt.grid(True)

    try:
        while not stop_event.is_set():
            try:
                xAvg, yAvg = data_queue.get(timeout=0.1)
                plotX.append(xAvg)
                plotY.append(yAvg)
                datax = np.array(plotX)
                datay = np.array(plotY)
                line1.set_xdata(datax)
                line1.set_ydata(datay)
                ax.relim()
                ax.autoscale_view()
                figure.canvas.draw()
                figure.canvas.flush_events()
            except queue.Empty:
                continue
    except KeyboardInterrupt:
        stop_event.set()

    plt.ioff()
    plt.show()
    aruco_thread.join()


Error: Tidak dapat membaca frame dari kamera.


: 

kode terbaik sejauh ini menurut chat gpt (udah ada pergerakan, pemrosesan posisi, sama rekam navdata)

In [2]:
import cv2
import threading
import queue
import time
from pyardrone import ARDrone, at
from time import sleep, perf_counter
import numpy as np
import os
import matplotlib
matplotlib.use('TkAgg')  # Pastikan menggunakan backend GUI yang tepat
import matplotlib.pyplot as plt

# Shared data and synchronization
class SharedData:
    def __init__(self):
        self.running_program = True
        self.setup_moving = False
        self.direction = 0
        self.last_time_move = perf_counter()
        self.navdata_records = [['current_time', 'battery', 'flying_state', 'altitude', 'vx', 'vy', 'vz', 'yaw', 'pitch', 'roll']]
        self.lock = threading.Lock()
        self.plotX = []
        self.plotY = []

shared_data = SharedData()

def movAvg(window, dataMovAvgX, dataMovAvgY, xEst, yEst):
    if len(dataMovAvgX) >= (window - 1):
        dataMovAvgX.append(xEst)
        dataMovAvgY.append(yEst)
        xAvg = (sum(dataMovAvgX) / len(dataMovAvgX))
        yAvg = (sum(dataMovAvgY) / len(dataMovAvgY))
        dataMovAvgX.pop(0)
        dataMovAvgY.pop(0)
    elif len(dataMovAvgX) < (window - 1):
        dataMovAvgX.append(xEst)
        dataMovAvgY.append(yEst)
        xAvg = xEst
        yAvg = yEst
    return xAvg, yAvg, dataMovAvgX, dataMovAvgY

def dataAruco(num_rows, num_cols, jarak):
    total_markers = num_rows * num_cols
    databasePos = []
    dataID = []
    i = 1
    for row in range(num_cols):
        for col in range(num_rows):
            posX = jarak * row
            posY = jarak * col
            databasePos.append([posX, posY])
            dataID.append(i)
            i += 1
    return dataID, databasePos

def saveImage(output_folder, file_counter, frame):
    filename = os.path.join(output_folder, f'detected_marker_{file_counter:04d}.png')
    cv2.imwrite(filename, frame)
    file_counter += 1
    return file_counter

def convertPos(image, ids, corners, dataID, databasePos):
    sumx = 0
    sumy = 0
    for i in range(len(ids)):
        sumx += databasePos[dataID.index(ids[i])][0]
        sumy += databasePos[dataID.index(ids[i])][1]

    x = sumx / len(ids)
    y = sumy / len(ids)

    tengah = []
    for i in range(len(ids)):
        xTengah = (int(corners[i][0][0][0]) + int(corners[i][0][1][0]) + int(corners[i][0][2][0]) + int(corners[i][0][3][0])) / 4
        yTengah = (int(corners[i][0][0][1]) + int(corners[i][0][1][1]) + int(corners[i][0][2][1]) + int(corners[i][0][3][1])) / 4
        tengah.append([xTengah, yTengah])

    sumtengahx = 0
    sumtengahy = 0
    for i in range(len(ids)):
        sumtengahx += tengah[i][0]
        sumtengahy += tengah[i][1]

    posPixelX = int(sumtengahx / len(ids))
    posPixelY = int(sumtengahy / len(ids))

    height, width = image.shape[:2]
    pusatKameraX = int(width / 2)
    pusatKameraY = int(height / 2)

    aX = 0.123
    aY = 0.131
    rXPixel = pusatKameraX - posPixelX
    rYPixel = pusatKameraY - posPixelY
    rX = rXPixel * aX
    rY = rYPixel * aY
    xEst = x + rX
    yEst = y + rY

    return x, y, posPixelX, posPixelY, xEst, yEst, pusatKameraX, pusatKameraY, tengah

def cameraDisplay(frame, corners, ids, posPixelX, posPixelY, pusatKameraX, pusatKameraY, tengah):
    uniqDisplay = False
    if ids is not None:
        radius = 5
        cv2.aruco.drawDetectedMarkers(frame, corners, ids)
        if uniqDisplay:
            for i in range(len(ids)):
                xTengah = tengah[i][0]
                yTengah = tengah[i][1]
                cv2.circle(frame, (int(xTengah), int(yTengah)), int(radius), (0, 0, 255), -1)
            cv2.circle(frame, (posPixelX, posPixelY), int(2 * radius), (0, 255, 0), -1)
            cv2.circle(frame, (pusatKameraX, pusatKameraY), int(2 * radius), (255, 0, 0), -1)

    cv2.imshow('Detected ArUco Markers', frame)

id_min1 = []
ids = []
tengah = []
dataAkhir = [["Waktu", "X_movAvg", "y_movAvg", "Pusat_x", "Pusat_y", "MFP_X", "MFP_Y", "Pixel_pusat_X", "Pixel_pusat_Y", "Pixel_MFP_X", "Pixel_MFP_Y"]]
num_rows = 25
num_cols = 20
id_1 = []
window = 20
dataMovAvgX = []
dataMovAvgY = []

output_folder = 'detected_frames'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

dataID, databasePos = dataAruco(num_rows, num_cols, 19.5)

file_counter = 0

# Function to send commands to the drone
def send_commands(drone, kec, waktu_move, shared_data):
    try:
        while shared_data.running_program:
            current_time = perf_counter()
            with shared_data.lock:
                if not shared_data.setup_moving:
                    takeoff(drone)
                    sleep(5)
                    print("Drone Take off")
                    shared_data.setup_moving = True

                if (current_time - shared_data.last_time_move) >= waktu_move:
                    if shared_data.direction == 0:
                        shared_data.direction = 1
                        print("Drone Maju")
                    elif shared_data.direction == 1:
                        hover(drone)
                        shared_data.direction = 2
                        print("Drone Ke kanan")
                    elif shared_data.direction == 2:
                        hover(drone)
                        shared_data.direction = 3
                        print("Drone ke belakang")
                    elif shared_data.direction == 3:
                        hover(drone)
                        shared_data.direction = 4
                        print("Drone ke kiri")
                    elif shared_data.direction == 4:
                        hover(drone)
                        shared_data.direction = 5
                        print("Trajektori selesai, akan mendarat")

                    shared_data.last_time_move = current_time

            if shared_data.setup_moving:
                if shared_data.direction == 1:
                    move(drone, forward=kec)
                elif shared_data.direction == 2:
                    move(drone, right=kec)
                elif shared_data.direction == 3:
                    move(drone, backward=kec)
                elif shared_data.direction == 4:
                    move(drone, left=kec)
                elif shared_data.direction == 5:
                    land(drone)
                    print("Drone Mendarat")
                    shared_data.running_program = False

    except Exception as e:
        print(f"Error during command sending: {e}")
        land(drone)
        print("Drone mendarat")

# Function to read the video stream from the drone and process ArUco markers
def read_video_stream(shared_data):
    try:
        ardrone_url = 'tcp://192.168.1.1:5555'
        cap = cv2.VideoCapture(ardrone_url)

        if not cap.isOpened():
            print("Error: Tidak dapat membuka kamera.")
            return

        aruco_dict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_6X6_1000)
        aruco_params = cv2.aruco.DetectorParameters()

        while shared_data.running_program:
            current_time = perf_counter()
            ret, frame = cap.read()

            if not ret:
                print("Error: Tidak dapat membaca frame dari kamera.")
                break

            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            corners, ids, rejected = cv2.aruco.detectMarkers(gray, aruco_dict, parameters=aruco_params)

            if ids is not None:
                x, y, posPixelX, posPixelY, xEst, yEst, pusatKameraX, pusatKameraY, tengah = convertPos(frame, ids, corners, dataID, databasePos)
                xAvg, yAvg, dataMovAvgX, dataMovAvgY = movAvg(window, dataMovAvgX, dataMovAvgY, xEst, yEst)
                dataAkhir.append([current_time, xAvg, yAvg, xEst, yEst, x, y, pusatKameraX, pusatKameraY, posPixelX, posPixelY])

                with shared_data.lock:
                    shared_data.plotX.append(xAvg)
                    shared_data.plotY.append(yAvg)

                if (len(id_1) != len(ids)) or ((len(id_1) == len(ids)) and (id_1[0] != ids[0])):
                    id_1 = ids

            cameraDisplay(frame, corners, ids, posPixelX, posPixelY, pusatKameraX, pusatKameraY, tengah)

            if cv2.waitKey(1) & 0xFF == ord('q'):
                shared_data.running_program = False
                break

        cap.release()
        cv2.destroyAllWindows()
        np.savetxt("Data_Akhir.txt", dataAkhir, fmt="%s")

    except Exception as e:
        print(f"Error during video streaming: {e}")

def live_plotting(shared_data):
    plotX, plotY = [], []

    plt.ion()
    figure, ax = plt.subplots(figsize=(10, 8))
    line1, = ax.plot(plotX, plotY, color='blue', marker='.', linewidth=0.5, markersize=15)
    ax.set_xlim(0, 400)
    ax.set_ylim(0, 500)
    plt.xlabel("X-axis")
    plt.ylabel("Y-axis")
    plt.title("Updating plot...")

    while shared_data.running_program:
        with shared_data.lock:
            plotX = shared_data.plotX.copy()
            plotY = shared_data.plotY.copy()

        line1.set_xdata(plotX)
        line1.set_ydata(plotY)

        ax.relim()
        ax.autoscale_view()
        figure.canvas.draw()
        figure.canvas.flush_events()
        sleep(0.1)

    plt.ioff()
    plt.show()

# Function to retrieve navigation data
def get_navdata(drone, navdemo, sending_state, shared_data):
    try:
        while shared_data.running_program:
            if not sending_state:
                if drone is not None:
                    if navdemo is not None:
                        if not drone.state.navdata_bootstrap:
                            navdemo = drone.navdata.demo
                            vx = navdemo.vx / 1000
                            vy = navdemo.vy / 1000
                            vz = navdemo.vz / 1000
                            yaw = navdemo.psi / 1000
                            pitch = navdemo.theta / 1000
                            roll = navdemo.phi / 1000
                            altitude = navdemo.altitude / 1000
                            flying_state = drone.state.fly_mask
                            battery = navdemo.vbat_flying_percentage
                            with shared_data.lock:
                                shared_data.navdata_records.append([perf_counter(), battery, flying_state, altitude, vx, vy, vz, yaw, pitch, roll])
                    else:
                        if not drone.state.navdata_bootstrap:
                            navdemo = drone.navdata.demo
                        else:
                            print("navdata demo is problem")
                            drone.send(at.CONFIG('general:navdata_demo', True))
            sleep(0.1)  # Adjust the sleep interval as needed
    except Exception as e:
        print(f"Error during navdata retrieval: {e}")

# Function to connect to the drone
def connect(drone, ip):
    try:
        if drone is None:
            drone = ARDrone(host=ip)
            drone.send(at.CONFIG('general:navdata_demo', True))
            connected = True
        else:
            if not drone.state.navdata_bootstrap:
                navdemo = drone.navdata.demo
                print("connected")
                connected = True
            else:
                print("navdata demo is problem")
                drone.send(at.CONFIG('general:navdata_demo', True))
                sleep(1.5)
                navdemo = drone.navdata.demo
                connected = False
        sleep(0.5)
        return drone, connected
    except Exception as e:
        print(f"Error during connection: {e}")
        return None, False

# Function to disconnect from the drone
def disconnect(drone):
    try:
        drone.close()
        return None
    except Exception as e:
        print(f"Error during disconnection: {e}")
        return drone

# Function to move the drone
def move(drone, forward=0.0, backward=0.0, right=0.0, left=0.0, up=0.0, down=0.0, cw=0.0, ccw=0.0):
    try:
        if drone is not None:
            drone.move(forward=forward, backward=backward, right=right, left=left, up=up, down=down, cw=cw, ccw=ccw)
        else:
            print("Drone belum terhubung")
    except Exception as e:
        print(f"Error during drone movement: {e}")

# Function to hover the drone
def hover(drone):
    try:
        if drone is not None:
            drone.move(forward=0.0, backward=0.0, right=0.0, left=0.0, up=0.0, down=0.0, cw=0.0, ccw=0.0)
            print('drone berhasil hover')
            sleep(1)
        else:
            print("Drone belum terhubung")
    except Exception as e:
        print(f"Error during hover: {e}")

# Function to take off the drone
def takeoff(drone):
    try:
        print('takeoff')
        drone.takeoff()
    except Exception as e:
        print(f"Error during takeoff: {e}")

# Function to land the drone
def land(drone):
    try:
        print('landing')
        drone.land()
    except Exception as e:
        print(f"Error during landing: {e}")

# Function to trim the drone
def trim(drone):
    try:
        drone.send(at.FTRIM())
        sleep(2)
    except Exception as e:
        print(f"Error during trim: {e}")

# Function to calibrate the drone
def calib(drone):
    try:
        drone.send(at.CALIB(0))
    except Exception as e:
        print(f"Error during calibration: {e}")

# Function to change the drone's camera
def change_camera(drone, camera_condition):
    try:
        drone.send(at.CONFIG("video:video_channel", camera_condition))
        return camera_condition
    except Exception as e:
        print(f"Error during camera change: {e}")


# Initialize variables
drone = None
ip = '192.168.1.1'
kec = 0.1
waktu_move = 3

# Control flag for navdata retrieval
enable_navdata = False

# Connect to the drone
drone, connected = connect(drone, ip)

if drone is not None:
    try:
        # Start command thread
        command_thread = threading.Thread(target=send_commands, args=(drone, kec, waktu_move, shared_data))
        video_thread = threading.Thread(target=read_video_stream, args=(shared_data,))
        plot_thread = threading.Thread(target=live_plotting, args=(shared_data,))
        # Conditionally start navdata thread
        if enable_navdata:
            navdata_thread = threading.Thread(target=get_navdata, args=(drone, None, False, shared_data))

        # Mulai thread 
        command_thread.start()
        video_thread.start()
        plot_thread.start()
        # Conditionally start navdata thread
        if enable_navdata:
            navdata_thread.start()

        # Wait for threads to finish
        command_thread.join()
        video_thread.join()
        plot_thread.join()
        if enable_navdata:
            navdata_thread.join()
        

    except KeyboardInterrupt:
        print("Program dihentikan oleh pengguna. Drone akan mendarat")
        shared_data.running_program = False
        land(drone)
        print("Drone mendarat")

    except Exception as e:
        print(f"Unexpected error: {e}")
        shared_data.running_program = False
        land(drone)
        print("Drone mendarat")

    finally:
        np.savetxt("testDataArray.txt", shared_data.navdata_records, fmt="%s")
        drone = disconnect(drone)
else:
    print("Tidak dapat terhubung ke drone.")


takeoff


C:\Users\ACER\AppData\Local\Temp\ipykernel_19024\578434023.py:243: UserWarning: Starting a Matplotlib GUI outside of the main thread will likely fail.
  figure, ax = plt.subplots(figsize=(10, 8))


Drone Take off
Drone Maju
drone berhasil hover
Drone Ke kanan
drone berhasil hover
Drone ke belakang
drone berhasil hover
Drone ke kiri
drone berhasil hover
Trajektori selesai, akan mendarat
landing
Drone Mendarat


C:\Users\ACER\AppData\Local\Temp\ipykernel_19024\578434023.py:266: UserWarning: Starting a Matplotlib GUI outside of the main thread will likely fail.
  plt.show()


In [5]:
import cv2
import threading
import queue
import time
from pyardrone import ARDrone, at
from time import sleep, perf_counter
import numpy as np
import os
# import matplotlib
# matplotlib.use('TkAgg')  # Pastikan menggunakan backend GUI yang tepat
# import matplotlib.pyplot as plt

# Shared data and synchronization
class SharedData:
    def __init__(self):
        self.running_program = True
        self.setup_moving = False
        self.direction = 0
        self.last_time_move = perf_counter()
        self.navdata_records = [['current_time', 'battery', 'flying_state', 'altitude', 'vx', 'vy', 'vz', 'yaw', 'pitch', 'roll']]
        self.lock = threading.Lock()
        self.plotX = []
        self.plotY = []

shared_data = SharedData()

def movAvg(window, dataMovAvgX, dataMovAvgY, xEst, yEst):
    if len(dataMovAvgX) >= (window - 1):
        dataMovAvgX.append(xEst)
        dataMovAvgY.append(yEst)
        xAvg = (sum(dataMovAvgX) / len(dataMovAvgX))
        yAvg = (sum(dataMovAvgY) / len(dataMovAvgY))
        dataMovAvgX.pop(0)
        dataMovAvgY.pop(0)
    elif len(dataMovAvgX) < (window - 1):
        dataMovAvgX.append(xEst)
        dataMovAvgY.append(yEst)
        xAvg = xEst
        yAvg = yEst
    return xAvg, yAvg, dataMovAvgX, dataMovAvgY

def dataAruco(num_rows, num_cols, jarak):
    total_markers = num_rows * num_cols
    databasePos = []
    dataID = []
    i = 1
    for row in range(num_cols):
        for col in range(num_rows):
            posX = jarak * row
            posY = jarak * col
            databasePos.append([posX, posY])
            dataID.append(i)
            i += 1
    return dataID, databasePos

def saveImage(output_folder, file_counter, frame):
    filename = os.path.join(output_folder, f'detected_marker_{file_counter:04d}.png')
    cv2.imwrite(filename, frame)
    file_counter += 1
    return file_counter

def convertPos(image, ids, corners, dataID, databasePos):
    sumx = 0
    sumy = 0
    for i in range(len(ids)):
        sumx += databasePos[dataID.index(ids[i])][0]
        sumy += databasePos[dataID.index(ids[i])][1]

    x = sumx / len(ids)
    y = sumy / len(ids)

    tengah = []
    for i in range(len(ids)):
        xTengah = (int(corners[i][0][0][0]) + int(corners[i][0][1][0]) + int(corners[i][0][2][0]) + int(corners[i][0][3][0])) / 4
        yTengah = (int(corners[i][0][0][1]) + int(corners[i][0][1][1]) + int(corners[i][0][2][1]) + int(corners[i][0][3][1])) / 4
        tengah.append([xTengah, yTengah])

    sumtengahx = 0
    sumtengahy = 0
    for i in range(len(ids)):
        sumtengahx += tengah[i][0]
        sumtengahy += tengah[i][1]

    posPixelX = int(sumtengahx / len(ids))
    posPixelY = int(sumtengahy / len(ids))

    height, width = image.shape[:2]
    pusatKameraX = int(width / 2)
    pusatKameraY = int(height / 2)

    aX = 0.123
    aY = 0.131
    rXPixel = pusatKameraX - posPixelX
    rYPixel = pusatKameraY - posPixelY
    rX = rXPixel * aX
    rY = rYPixel * aY
    xEst = x + rX
    yEst = y + rY

    return x, y, posPixelX, posPixelY, xEst, yEst, pusatKameraX, pusatKameraY, tengah

def cameraDisplay(frame, corners, ids, posPixelX, posPixelY, pusatKameraX, pusatKameraY, tengah):
    uniqDisplay = False
    if ids is not None:
        radius = 5
        cv2.aruco.drawDetectedMarkers(frame, corners, ids)
        if uniqDisplay:
            for i in range(len(ids)):
                xTengah = tengah[i][0]
                yTengah = tengah[i][1]
                cv2.circle(frame, (int(xTengah), int(yTengah)), int(radius), (0, 0, 255), -1)
            cv2.circle(frame, (posPixelX, posPixelY), int(2 * radius), (0, 255, 0), -1)
            cv2.circle(frame, (pusatKameraX, pusatKameraY), int(2 * radius), (255, 0, 0), -1)

    cv2.imshow('Detected ArUco Markers', frame)

id_min1 = []
ids = []
tengah = []
dataAkhir = [["Waktu", "X_movAvg", "y_movAvg", "Pusat_x", "Pusat_y", "MFP_X", "MFP_Y", "Pixel_pusat_X", "Pixel_pusat_Y", "Pixel_MFP_X", "Pixel_MFP_Y"]]
num_rows = 25
num_cols = 20
id_1 = []
window = 20
dataMovAvgX = []
dataMovAvgY = []

output_folder = 'detected_frames'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

dataID, databasePos = dataAruco(num_rows, num_cols, 19.5)

file_counter = 0

# Function to connect to the drone
def connect(drone, ip):
    try:
        if drone is None:
            drone = ARDrone(host=ip)
            drone.send(at.CONFIG('general:navdata_demo', True))
            connected = True
        else:
            if not drone.state.navdata_bootstrap:
                navdemo = drone.navdata.demo
                print("connected")
                connected = True
            else:
                print("navdata demo is problem")
                drone.send(at.CONFIG('general:navdata_demo', True))
                sleep(1.5)
                navdemo = drone.navdata.demo
                connected = False
        sleep(0.5)
        return drone, connected
    except Exception as e:
        print(f"Error during connection: {e}")
        return None, False

# Function to disconnect from the drone
def disconnect(drone):
    try:
        drone.close()
        return None
    except Exception as e:
        print(f"Error during disconnection: {e}")
        return drone

# Function to move the drone
def move(drone, forward=0.0, backward=0.0, right=0.0, left=0.0, up=0.0, down=0.0, cw=0.0, ccw=0.0):
    try:
        if drone is not None:
            drone.move(forward=forward, backward=backward, right=right, left=left, up=up, down=down, cw=cw, ccw=ccw)
        else:
            print("Drone belum terhubung")
    except Exception as e:
        print(f"Error during drone movement: {e}")

# Function to hover the drone
def hover(drone):
    try:
        if drone is not None:
            drone.move(forward=0.0, backward=0.0, right=0.0, left=0.0, up=0.0, down=0.0, cw=0.0, ccw=0.0)
            print('drone berhasil hover')
            sleep(1)
        else:
            print("Drone belum terhubung")
    except Exception as e:
        print(f"Error during hover: {e}")

# Function to take off the drone
def takeoff(drone):
    try:
        print('takeoff')
        drone.takeoff()
    except Exception as e:
        print(f"Error during takeoff: {e}")

# Function to land the drone
def land(drone):
    try:
        print('landing')
        drone.land()
    except Exception as e:
        print(f"Error during landing: {e}")

# Function to trim the drone
def trim(drone):
    try:
        drone.send(at.FTRIM())
        sleep(2)
    except Exception as e:
        print(f"Error during trim: {e}")

# Function to calibrate the drone
def calib(drone):
    try:
        drone.send(at.CALIB(0))
    except Exception as e:
        print(f"Error during calibration: {e}")

# Function to change the drone's camera
def change_camera(drone, camera_condition):
    try:
        drone.send(at.CONFIG("video:video_channel", camera_condition))
        return camera_condition
    except Exception as e:
        print(f"Error during camera change: {e}")

# Function to send commands to the drone
def send_commands(drone, kec, waktu_move, shared_data):
    try:
        while shared_data.running_program:
            current_time = perf_counter()
            with shared_data.lock:
                if not shared_data.setup_moving:
                    takeoff(drone)
                    sleep(5)
                    print("Drone Take off")
                    shared_data.setup_moving = True

                if (current_time - shared_data.last_time_move) >= waktu_move:
                    if shared_data.direction == 0:
                        shared_data.direction = 1
                        print("Drone Maju")
                    elif shared_data.direction == 1:
                        hover(drone)
                        shared_data.direction = 2
                        print("Drone Ke kanan")
                    elif shared_data.direction == 2:
                        hover(drone)
                        shared_data.direction = 3
                        print("Drone ke belakang")
                    elif shared_data.direction == 3:
                        hover(drone)
                        shared_data.direction = 4
                        print("Drone ke kiri")
                    elif shared_data.direction == 4:
                        hover(drone)
                        shared_data.direction = 5
                        print("Trajektori selesai, akan mendarat")

                    shared_data.last_time_move = current_time

            if shared_data.setup_moving:
                if shared_data.direction == 1:
                    move(drone, forward=kec)
                elif shared_data.direction == 2:
                    move(drone, right=kec)
                elif shared_data.direction == 3:
                    move(drone, backward=kec)
                elif shared_data.direction == 4:
                    move(drone, left=kec)
                elif shared_data.direction == 5:
                    land(drone)
                    print("Drone Mendarat")
                    shared_data.running_program = False

    except Exception as e:
        print(f"Error during command sending: {e}")
        land(drone)
        print("Drone mendarat")

# Function to read the video stream from the drone and process ArUco markers
def read_video_stream(shared_data):
    try:
        ardrone_url = 'tcp://192.168.1.1:5555'
        cap = cv2.VideoCapture(ardrone_url)

        if not cap.isOpened():
            print("Error: Tidak dapat membuka kamera.")
            return

        aruco_dict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_6X6_1000)
        aruco_params = cv2.aruco.DetectorParameters()

        while shared_data.running_program:
            current_time = perf_counter()
            ret, frame = cap.read()

            if not ret:
                print("Error: Tidak dapat membaca frame dari kamera.")
                break

            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            corners, ids, rejected = cv2.aruco.detectMarkers(gray, aruco_dict, parameters=aruco_params)

            if ids is not None:
                x, y, posPixelX, posPixelY, xEst, yEst, pusatKameraX, pusatKameraY, tengah = convertPos(frame, ids, corners, dataID, databasePos)
                xAvg, yAvg, dataMovAvgX, dataMovAvgY = movAvg(window, dataMovAvgX, dataMovAvgY, xEst, yEst)
                dataAkhir.append([current_time, xAvg, yAvg, xEst, yEst, x, y, pusatKameraX, pusatKameraY, posPixelX, posPixelY])

                with shared_data.lock:
                    shared_data.plotX.append(xAvg)
                    shared_data.plotY.append(yAvg)

                if (len(id_1) != len(ids)) or ((len(id_1) == len(ids)) and (id_1[0] != ids[0])):
                    id_1 = ids

            cameraDisplay(frame, corners, ids, posPixelX, posPixelY, pusatKameraX, pusatKameraY, tengah)

            if cv2.waitKey(1) & 0xFF == ord('q'):
                shared_data.running_program = False
                break

        cap.release()
        cv2.destroyAllWindows()
        np.savetxt("Data_Akhir.txt", dataAkhir, fmt="%s")

    except Exception as e:
        print(f"Error during video streaming: {e}")

# Function to retrieve navigation data
def get_navdata(drone, navdemo, sending_state, shared_data):
    try:
        while shared_data.running_program:
            if not sending_state:
                if drone is not None:
                    if navdemo is not None:
                        if not drone.state.navdata_bootstrap:
                            navdemo = drone.navdata.demo
                            vx = navdemo.vx / 1000
                            vy = navdemo.vy / 1000
                            vz = navdemo.vz / 1000
                            yaw = navdemo.psi / 1000
                            pitch = navdemo.theta / 1000
                            roll = navdemo.phi / 1000
                            altitude = navdemo.altitude / 1000
                            flying_state = drone.state.fly_mask
                            battery = navdemo.vbat_flying_percentage
                            with shared_data.lock:
                                shared_data.navdata_records.append([perf_counter(), battery, flying_state, altitude, vx, vy, vz, yaw, pitch, roll])
                    else:
                        if not drone.state.navdata_bootstrap:
                            navdemo = drone.navdata.demo
                        else:
                            print("navdata demo is problem")
                            drone.send(at.CONFIG('general:navdata_demo', True))
            sleep(0.1)  # Adjust the sleep interval as needed
    except Exception as e:
        print(f"Error during navdata retrieval: {e}")


# Initialize variables
drone = None
ip = '192.168.1.1'
kec = 0.1
waktu_move = 3

# Control flag for navdata retrieval
enable_navdata = False

# Connect to the drone
drone, connected = connect(drone, ip)

if drone is not None:
    try:
        # Start command thread
        command_thread = threading.Thread(target=send_commands, args=(drone, kec, waktu_move, shared_data))
        video_thread = threading.Thread(target=read_video_stream, args=(shared_data,))
        # Conditionally start navdata thread
        if enable_navdata:
            navdata_thread = threading.Thread(target=get_navdata, args=(drone, None, False, shared_data))

        # Mulai thread 
        command_thread.start()
        video_thread.start()
        # Conditionally start navdata thread
        if enable_navdata:
            navdata_thread.start()

        # Wait for threads to finish
        command_thread.join()
        video_thread.join()
        if enable_navdata:
            navdata_thread.join()
        

    except KeyboardInterrupt:
        print("Program dihentikan oleh pengguna. Drone akan mendarat")
        shared_data.running_program = False
        land(drone)
        print("Drone mendarat")

    except Exception as e:
        print(f"Unexpected error: {e}")
        shared_data.running_program = False
        land(drone)
        print("Drone mendarat")

    finally:
        np.savetxt("testDataArray.txt", shared_data.navdata_records, fmt="%s")
        drone = disconnect(drone)
else:
    print("Tidak dapat terhubung ke drone.")

takeoff
Drone Take off
Drone Maju
drone berhasil hover
Drone Ke kanan
drone berhasil hover
Drone ke belakang
drone berhasil hover
Drone ke kiri
drone berhasil hover
Trajektori selesai, akan mendarat
landing
Drone Mendarat


In [2]:
import cv2 as cv
import threading
import time
from pyardrone import ARDrone, at
from time import sleep, perf_counter
import numpy as np

# Shared data and synchronization
class SharedData:
    def __init__(self):
        self.running_program = True
        self.setup_moving = False
        self.direction = 0
        self.last_time_move = perf_counter()
        self.navdata_records = [['current_time', 'battery', 'flying_state', 'altitude', 'vx', 'vy', 'vz', 'yaw', 'pitch', 'roll']]
        self.lock = threading.Lock()

shared_data = SharedData()

# Function to send commands to the drone
def send_commands(drone, kec, waktu_move, shared_data):
    try:
        while shared_data.running_program:
            current_time = perf_counter()
            with shared_data.lock:
                if not shared_data.setup_moving:
                    takeoff(drone)
                    sleep(5)
                    print("Drone Take off")
                    shared_data.setup_moving = True

                if (current_time - shared_data.last_time_move) >= waktu_move:
                    if shared_data.direction == 0:
                        shared_data.direction = 1
                        print("Drone Maju")
                    elif shared_data.direction == 1:
                        hover(drone)
                        shared_data.direction = 2
                        print("Drone Ke kanan")
                    elif shared_data.direction == 2:
                        hover(drone)
                        shared_data.direction = 3
                        print("Drone ke belakang")
                    elif shared_data.direction == 3:
                        hover(drone)
                        shared_data.direction = 4
                        print("Drone ke kiri")
                    elif shared_data.direction == 4:
                        hover(drone)
                        shared_data.direction = 5
                        print("Trajektori selesai, akan mendarat")

                    shared_data.last_time_move = current_time

            if shared_data.setup_moving:
                if shared_data.direction == 1:
                    move(drone, forward=kec)
                elif shared_data.direction == 2:
                    move(drone, right=kec)
                elif shared_data.direction == 3:
                    move(drone, backward=kec)
                elif shared_data.direction == 4:
                    move(drone, left=kec)
                elif shared_data.direction == 5:
                    land(drone)
                    print("Drone Mendarat")
                    shared_data.running_program = False

    except Exception as e:
        print(f"Error during command sending: {e}")
        land(drone)
        print("Drone mendarat")

# Function to read the video stream from the drone
def read_video_stream(shared_data):
    try:
        ardrone_url = 'tcp://192.168.1.1:5555'
        camera = cv.VideoCapture(ardrone_url)
        
        if not camera.isOpened():
            print("Gagal membuka stream video. Pastikan URL benar dan drone terhubung.")
            shared_data.running_program = False
            return

        while shared_data.running_program:
            ret, frame = camera.read()
            if ret:
                cv.imshow('Drone Camera', frame)
                if cv.waitKey(1) & 0xFF == ord('q'):
                    shared_data.running_program = False
                    break
            else:
                print("Tidak ada gambar tertangkap atau stream terputus.")
                shared_data.running_program = False
                break

        camera.release()
        cv.destroyAllWindows()

    except Exception as e:
        print(f"Error during video streaming: {e}")

# Function to retrieve navigation data
def get_navdata(drone, navdemo, sending_state, shared_data):
    try:
        while shared_data.running_program:
            if not sending_state:
                if drone is not None:
                    if navdemo is not None:
                        if not drone.state.navdata_bootstrap:
                            navdemo = drone.navdata.demo
                            vx = navdemo.vx / 1000
                            vy = navdemo.vy / 1000
                            vz = navdemo.vz / 1000
                            yaw = navdemo.psi / 1000
                            pitch = navdemo.theta / 1000
                            roll = navdemo.phi / 1000
                            altitude = navdemo.altitude / 1000
                            flying_state = drone.state.fly_mask
                            battery = navdemo.vbat_flying_percentage
                            with shared_data.lock:
                                shared_data.navdata_records.append([perf_counter(), battery, flying_state, altitude, vx, vy, vz, yaw, pitch, roll])
                    else:
                        if not drone.state.navdata_bootstrap:
                            navdemo = drone.navdata.demo
                        else:
                            print("navdata demo is problem")
                            drone.send(at.CONFIG('general:navdata_demo', True))
            sleep(0.1)  # Adjust the sleep interval as needed
    except Exception as e:
        print(f"Error during navdata retrieval: {e}")

# Function to connect to the drone
def connect(drone, ip):
    try:
        if drone is None:
            drone = ARDrone(host=ip)
            drone.send(at.CONFIG('general:navdata_demo', True))
            connected = True
        else:
            if not drone.state.navdata_bootstrap:
                navdemo = drone.navdata.demo
                print("connected")
                connected = True
            else:
                print("navdata demo is problem")
                drone.send(at.CONFIG('general:navdata_demo', True))
                sleep(1.5)
                navdemo = drone.navdata.demo
                connected = False
        sleep(0.5)
        return drone, connected
    except Exception as e:
        print(f"Error during connection: {e}")
        return None, False

# Function to disconnect from the drone
def disconnect(drone):
    try:
        drone.close()
        return None
    except Exception as e:
        print(f"Error during disconnection: {e}")
        return drone

# Function to move the drone
def move(drone, forward=0.0, backward=0.0, right=0.0, left=0.0, up=0.0, down=0.0, cw=0.0, ccw=0.0):
    try:
        if drone is not None:
            drone.move(forward=forward, backward=backward, right=right, left=left, up=up, down=down, cw=cw, ccw=ccw)
        else:
            print("Drone belum terhubung")
    except Exception as e:
        print(f"Error during drone movement: {e}")

# Function to hover the drone
def hover(drone):
    try:
        if drone is not None:
            drone.move(forward=0.0, backward=0.0, right=0.0, left=0.0, up=0.0, down=0.0, cw=0.0, ccw=0.0)
            print('drone berhasil hover')
            sleep(1)
        else:
            print("Drone belum terhubung")
    except Exception as e:
        print(f"Error during hover: {e}")

# Function to take off the drone
def takeoff(drone):
    try:
        print('takeoff')
        drone.takeoff()
    except Exception as e:
        print(f"Error during takeoff: {e}")

# Function to land the drone
def land(drone):
    try:
        print('landing')
        drone.land()
    except Exception as e:
        print(f"Error during landing: {e}")

# Function to trim the drone
def trim(drone):
    try:
        drone.send(at.FTRIM())
        sleep(2)
    except Exception as e:
        print(f"Error during trim: {e}")

# Function to calibrate the drone
def calib(drone):
    try:
        drone.send(at.CALIB(0))
    except Exception as e:
        print(f"Error during calibration: {e}")

# Function to change the drone's camera
def change_camera(drone, camera_condition):
    try:
        drone.send(at.CONFIG("video:video_channel", camera_condition))
        return camera_condition
    except Exception as e:
        print(f"Error during camera change: {e}")

# Initialize variables
drone = None
ip = '192.168.1.1'
kec = 0.1
waktu_move = 2

# Control flag for navdata retrieval
enable_navdata = True

# Connect to the drone
drone, connected = connect(drone, ip)

if drone is not None:
    try:
        # Start command thread
        command_thread = threading.Thread(target=send_commands, args=(drone, kec, waktu_move, shared_data))
        command_thread.start()

        # Conditionally start navdata thread
        if enable_navdata:
            navdata_thread = threading.Thread(target=get_navdata, args=(drone, None, False, shared_data))
            navdata_thread.start()

        # Start video stream thread
        video_thread = threading.Thread(target=read_video_stream, args=(shared_data,))
        video_thread.start()

        # Wait for threads to finish
        command_thread.join()
        if enable_navdata:
            navdata_thread.join()
        video_thread.join()

    except KeyboardInterrupt:
        print("Program dihentikan oleh pengguna. Drone akan mendarat")
        shared_data.running_program = False
        land(drone)
        print("Drone mendarat")

    except Exception as e:
        print(f"Unexpected error: {e}")
        shared_data.running_program = False
        land(drone)
        print("Drone mendarat")

    finally:
        np.savetxt("testDataArray.txt", shared_data.navdata_records, fmt="%s")
        drone = disconnect(drone)
else:
    print("Tidak dapat terhubung ke drone.")


takeoff
Drone Take off
Drone Maju
drone berhasil hover
Drone Ke kanan
drone berhasil hover
Drone ke belakang
drone berhasil hover
Drone ke kiri
drone berhasil hover
Trajektori selesai, akan mendarat
landing
Drone Mendarat


In [1]:
import threading
import socket
import cv2
import numpy as np
import os
from time import sleep, perf_counter
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
from pyardrone import ARDrone, at

# Fungsi untuk mengirim perintah ke drone
def send_command(drone, direction, kec):
    while True:
        if direction == 1:
            drone.move(forward=kec)
        elif direction == 2:
            drone.move(right=kec)
        elif direction == 3:
            drone.move(backward=kec)
        elif direction == 4:
            drone.move(left=kec)
        else:
            drone.hover()
        sleep(0.033)

# Fungsi untuk menerima aliran video dan memproses gambar dari drone
def process_video(dataID, databasePos, window, output_folder):
    def movAvg(window, dataMovAvgX, dataMovAvgY, xEst, yEst):
        if len(dataMovAvgX) >= (window - 1):
            dataMovAvgX.append(xEst)
            dataMovAvgY.append(yEst)
            xAvg = (sum(dataMovAvgX) / len(dataMovAvgX))
            yAvg = (sum(dataMovAvgY) / len(dataMovAvgY))
            dataMovAvgX.pop(0)
            dataMovAvgY.pop(0)
        else:
            dataMovAvgX.append(xEst)
            dataMovAvgY.append(yEst)
            xAvg = xEst
            yAvg = yEst
        return xAvg, yAvg, dataMovAvgX, dataMovAvgY

    def saveImage(output_folder, file_counter, frame):
        filename = os.path.join(output_folder, f'detected_marker_{file_counter:04d}.png')
        cv2.imwrite(filename, frame)
        file_counter += 1
        return file_counter

    def convertPos(image, ids, corners, dataID, databasePos):
        sumx = 0
        sumy = 0
        for i in range(len(ids)):
            sumx += databasePos[dataID.index(ids[i])][0]
            sumy += databasePos[dataID.index(ids[i])][1]

        x = sumx / len(ids)
        y = sumy / len(ids)

        tengah = []
        for i in range(len(ids)):
            xTengah = (int(corners[i][0][0][0]) + int(corners[i][0][1][0]) + int(corners[i][0][2][0]) + int(corners[i][0][3][0])) / 4
            yTengah = (int(corners[i][0][0][1]) + int(corners[i][0][1][1]) + int(corners[i][0][2][1]) + int(corners[i][0][3][1])) / 4
            tengah.append([xTengah, yTengah])

        sumtengahx = 0
        sumtengahy = 0
        for i in range(len(ids)):
            sumtengahx += tengah[i][0]
            sumtengahy += tengah[i][1]

        posPixelX = int(sumtengahx / len(ids))
        posPixelY = int(sumtengahy / len(ids))

        height, width = image.shape[:2]
        pusatKameraX = int(width / 2)
        pusatKameraY = int(height / 2)

        aX = 0.123
        aY = 0.131
        rXPixel = pusatKameraX - posPixelX
        rYPixel = pusatKameraY - posPixelY
        rX = rXPixel * aX
        rY = rYPixel * aY
        xEst = x + rX
        yEst = y + rY

        return x, y, posPixelX, posPixelY, xEst, yEst, pusatKameraX, pusatKameraY, tengah

    def cameraDisplay(frame, corners, ids, posPixelX, posPixelY, pusatKameraX, pusatKameraY, tengah):
        if ids is not None:
            radius = 5
            cv2.aruco.drawDetectedMarkers(frame, corners, ids)
            for i in range(len(ids)):
                xTengah = tengah[i][0]
                yTengah = tengah[i][1]
                cv2.circle(frame, (int(xTengah), int(yTengah)), int(radius), (0, 0, 255), -1)
            cv2.circle(frame, (posPixelX, posPixelY), int(2 * radius), (0, 255, 0), -1)
            cv2.circle(frame, (pusatKameraX, pusatKameraY), int(2 * radius), (255, 0, 0), -1)

        cv2.imshow('Detected ArUco Markers', frame)
        return

    file_counter = 0
    dataMovAvgX = []
    dataMovAvgY = []
    plotX = []
    plotY = []
    posPixelX = None
    posPixelY = None
    pusatKameraX = None
    pusatKameraY = None
    tengah = []

    ardrone_url = 'tcp://192.168.1.1:5555'
    cap = cv2.VideoCapture(ardrone_url)

    if not cap.isOpened():
        print("Error: Tidak dapat membuka kamera.")
        exit()

    aruco_dict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_6X6_1000)
    aruco_params = cv2.aruco.DetectorParameters()

    while True:
        ret, frame = cap.read()
        if not ret:
            print("Error: Tidak dapat membaca frame dari kamera.")
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        corners, ids, rejected = cv2.aruco.detectMarkers(gray, aruco_dict, parameters=aruco_params)

        if ids is not None:
            x, y, posPixelX, posPixelY, xEst, yEst, pusatKameraX, pusatKameraY, tengah = convertPos(frame, ids, corners, dataID, databasePos)
            xAvg, yAvg, dataMovAvgX, dataMovAvgY = movAvg(window, dataMovAvgX, dataMovAvgY, xEst, yEst)
            plotX.append(xAvg)
            plotY.append(yAvg)

        cameraDisplay(frame, corners, ids, posPixelX, posPixelY, pusatKameraX, pusatKameraY, tengah)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()
    np.savetxt("Data_Akhir.txt", plotX, fmt="%s")

    plt.ion()
    figure, ax = plt.subplots(figsize=(10, 8))
    line1, = ax.plot(plotX, plotY, color='blue', marker='.', linewidth=0.5, markersize=15)
    ax.set_xlim(0, 400)
    ax.set_ylim(0, 500)
    plt.xlabel("X-axis")
    plt.ylabel("Y-axis")
    plt.title("Updating plot...")
    plt.grid(True)
    plt.ioff()
    plt.show(block=True)

# Fungsi untuk mengendalikan drone dan merekam navigation data
def control_drone(drone, navdata_records, kec):
    def connect(drone, ip):
        if drone is None:
            try:
                drone = ARDrone(host=ip)
                drone.send(at.CONFIG('general:navdata_demo', True))
                connected = True
            except:
                print("Can not connected")
                drone = None
                connected = False
        else:
            if drone.state.navdata_bootstrap is not True:
                navdemo = drone.navdata.demo
                print("Connected")
                connected = True
            else:
                print("Navdata demo is problem")
                drone.send(at.CONFIG('general:navdata_demo', True))
                sleep(1.5)
                navdemo = drone.navdata.demo
                connected = False
        sleep(0.5)
        return drone, connected

    def disconnect(drone):
        drone.close()
        drone = None
        return drone

    def get_navdata(drone, navdemo, current_time, navdata_records):
        if drone is not None:
            if navdemo is not None:
                if drone.state.navdata_bootstrap is not True:
                    navdemo = drone.navdata.demo
                    vx = navdemo.vx / 1000
                    vy = navdemo.vy / 1000
                    vz = navdemo.vz / 1000
                    yaw = navdemo.psi / 1000
                    pitch = navdemo.theta / 1000
                    roll = navdemo.phi / 1000
                    altitude = navdemo.altitude / 1000
                    flying_state = drone.state.fly_mask
                    battery = navdemo.vbat_flying_percentage
                    navdata_records.append([current_time, battery, flying_state, altitude, vx, vy, vz, yaw, pitch, roll])
            else:
                if drone.state.navdata_bootstrap is not True:
                    navdemo = drone.navdata.demo
        sleep(0.1)
        return navdemo, navdata_records

    drone = None
    connected = False
    while not connected:
        ip = "192.168.1.1"
        drone, connected = connect(drone, ip)
    drone.send(at.CONFIG('control:altitude_max', '3000'))
    sleep(0.1)
    drone.send(at.CONFIG('control:altitude_min', '500'))
    sleep(0.1)
    drone.send(at.CONFIG('detect:enemy_colors', '3'))
    sleep(0.1)
    drone.send(at.CONFIG('detect:detect_type', '3'))
    sleep(0.1)
    drone.send(at.CONFIG('control:outdoor', 'FALSE'))
    sleep(0.1)
    drone.send(at.CONFIG('control:flight_without_shell', 'FALSE'))
    sleep(0.1)

    drone.send(at.REF(at.REF.takeoff))
    sleep(0.5)
    drone.send(at.PCMD(False, 0, 0, 0, 0))
    sleep(3.5)

    end_time = perf_counter() + 30
    navdemo = None

    while perf_counter() < end_time:
        current_time = perf_counter()
        navdemo, navdata_records = get_navdata(drone, navdemo, current_time, navdata_records)
        send_command(drone, direction, kec)

    drone.send(at.REF(at.REF.landing))
    drone = disconnect(drone)

    np.savetxt("navdata_records.txt", navdata_records, fmt="%s")

# Main program
if __name__ == "__main__":
    dataID = [0, 1, 2]
    databasePos = [[0, 0], [1, 0], [2, 0]]
    window = 5
    output_folder = 'output_images'

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    navdata_records = []

    kec = 0.1
    direction = 1  # arah pergerakan

    video_thread = threading.Thread(target=process_video, args=(dataID, databasePos, window, output_folder))
    control_thread = threading.Thread(target=control_drone, args=(None, navdata_records, kec))

    video_thread.start()
    control_thread.start()

    video_thread.join()
    control_thread.join()


Exception in thread Thread-6 (control_drone):
Traceback (most recent call last):
  File "c:\Python310\lib\threading.py", line 1009, in _bootstrap_inner
    self.run()
  File "c:\Python310\lib\threading.py", line 946, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\ACER\AppData\Local\Temp\ipykernel_12768\23398393.py", line 231, in control_drone
AttributeError: type object 'REF' has no attribute 'takeoff'
C:\Users\ACER\AppData\Local\Temp\ipykernel_12768\23398393.py:150: UserWarning: Starting a Matplotlib GUI outside of the main thread will likely fail.
  figure, ax = plt.subplots(figsize=(10, 8))
C:\Users\ACER\AppData\Local\Temp\ipykernel_12768\23398393.py:159: UserWarning: Starting a Matplotlib GUI outside of the main thread will likely fail.
  plt.show(block=True)


Exception in thread Thread-9 (_video_client_job):
Traceback (most recent call last):
  File "c:\Python310\lib\threading.py", line 1009, in _bootstrap_inner
    self.run()
  File "c:\Python310\lib\threading.py", line 946, in run
    self._target(*self._args, **self._kwargs)
  File "c:\Python310\lib\site-packages\pyardrone\video.py", line 103, in _video_client_job
    data = rsock.recv(4096)
ConnectionAbortedError: [WinError 10053] An established connection was aborted by the software in your host machine


In [ ]:
def video_processing_thread():
    id_min1 = []
    ids = []
    tengah = []
    dataAkhir = [["Waktu", "X_movAvg", "y_movAvg", "Pusat_x", "Pusat_y", "MFP_X", "MFP_Y", "Pixel_pusat_X", "Pixel_pusat_Y", "Pixel_MFP_X", "Pixel_MFP_Y"]]
    num_rows = 25
    num_cols = 20
    id_1 = []
    window = 20
    dataMovAvgX = []
    dataMovAvgY = []
    plotX = []
    plotY = []
    posPixelX = None
    posPixelY = None
    pusatKameraX = None
    pusatKameraY = None
    tengah = []

    output_folder = 'detected_frames'
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    dataID, databasePos = dataAruco(num_rows, num_rows, 19.5)

    file_counter = 0
    init_time = perf_counter()
    current_time = 0

    ardrone_url = 'tcp://192.168.1.1:5555'
    cap = cv2.VideoCapture(ardrone_url)

    if not cap.isOpened():
        print("Error: Tidak dapat membuka kamera.")
        return

    aruco_dict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_6X6_1000)
    aruco_params = cv2.aruco.DetectorParameters()

    while True:
        current_time = perf_counter()
        ret, frame = cap.read()

        if not ret:
            print("Error: Tidak dapat membaca frame dari kamera.")
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        corners, ids, rejected = cv2.aruco.detectMarkers(gray, aruco_dict, parameters=aruco_params)

        if ids is not None:
            x, y, posPixelX, posPixelY, xEst, yEst, pusatKameraX, pusatKameraY, tengah = convertPos(frame, ids, corners, dataID, databasePos)
            xAvg, yAvg, dataMovAvgX, dataMovAvgY = movAvg(window, dataMovAvgX, dataMovAvgY, xEst, yEst)
            plotX.append(xAvg)
            plotY.append(yAvg)
            dataAkhir.append([current_time, xAvg, yAvg, xEst, yEst, x, y, pusatKameraX, pusatKameraY, posPixelX, posPixelY])
            if (len(id_1) != len(ids)) or ((len(id_1) == len(ids)) and (id_1[0] != ids[0])):
                id_1 = ids

        cameraDisplay(frame, corners, ids, posPixelX, posPixelY, pusatKameraX, pusatKameraY, tengah)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()
    np.savetxt("Data_Akhir.txt", dataAkhir, fmt="%s")

    plotDisplay = True

    if plotDisplay:
        plt.ion()
        figure, ax = plt.subplots(figsize=(10, 8))
        line1, = ax.plot(plotX, plotY, color='blue', marker='.', linewidth=0.5, markersize=15)
        ax.set_xlim(0, 400)
        ax.set_ylim(0, 500)
        plt.xlabel("X-axis")
        plt.ylabel("Y-axis")
        plt.title("Updating plot...")
        plt.grid(True)
        plt.ioff()
        plt.show(block=True)

# Bagian Code 2

def drone_control_thread():
    ip = "192.168.1.1"
    drone = None
    navdemo = None
    drone, navdemo, connected = connect(drone, ip, navdemo)
    time_t = 0
    time_s = 0
    n = 0
    first_loop = 1

    if connected:
        print("Battery: ", navdemo.vbat_flying_percentage, "%")
        time_s = 1.5
        drone.send(at.REF(at.REF_GROUND))
        while time_t <= 20:
            print("time: ", time_t, " seconds")
            if first_loop:
                drone.send(at.REF(at.REF_EMERGENCY))
                sleep(1)
                first_loop = 0
            else:
                if 0 < n <= 4:
                    print("Flying Up")
                    drone.send(at.PCMD(1, 0.0, 0.0, 0.1, 0.0))
                    sleep(1.0)
                elif 4 < n <= 6:
                    print("Hovering")
                    drone.send(at.PCMD(1, 0.0, 0.0, 0.0, 0.0))
                    sleep(1.0)
                elif 6 < n <= 8:
                    print("Rotate Left")
                    drone.send(at.PCMD(1, 0.0, 0.0, 0.0, -0.2))
                    sleep(1.0)
                elif 8 < n <= 10:
                    print("Hovering")
                    drone.send(at.PCMD(1, 0.0, 0.0, 0.0, 0.0))
                    sleep(1.0)
                elif 10 < n <= 14:
                    print("Move Forward")
                    drone.send(at.PCMD(1, 0.0, 0.5, 0.0, 0.0))
                    sleep(1.0)
                elif 14 < n <= 16:
                    print("Hovering")
                    drone.send(at.PCMD(1, 0.0, 0.0, 0.0, 0.0))
                    sleep(1.0)
                elif 16 < n <= 18:
                    print("Rotate Right")
                    drone.send(at.PCMD(1, 0.0, 0.0, 0.0, 0.2))
                    sleep(1.0)
                elif 18 < n <= 20:
                    print("Hovering")
                    drone.send(at.PCMD(1, 0.0, 0.0, 0.0, 0.0))
                    sleep(1.0)
            n += 1
            time_t += time_s

        print("Landing")
        drone.send(at.REF(at.REF_LANDING))
        sleep(1.0)
        drone.close()
    else:
        print("Drone not connected, exiting...")

# Main function to start threads

def main():
    video_thread = threading.Thread(target=video_processing_thread)
    control_thread = threading.Thread(target=drone_control_thread)

    video_thread.start()
    control_thread.start()

    video_thread.join()
    control_thread.join()

if __name__ == "__main__":
    main()


In [ ]:
def move_to_target(drone, xAvg, yAvg, target_x, target_y, first, error_x1, error_y1):
    """
    Fungsi untuk menggerakkan drone menuju target_x dan target_y berdasarkan posisi estimasi xAvg dan yAvg.
    """
    error_x = target_x - xAvg
    error_y = target_y - yAvg

    # print("error x = "+str(error_x)+" , error y = "+str(error_y))

    if first == True :
        error_x1 = error_x
        error_y1 = error_y
        first = False

    deltaErrorX = error_x-error_x1
    deltaErrorY = error_y-error_y1

    kec = 0.05
    kp = 0.5
    if (error_x > 0) and (error_x1 >= 0):
        right = kec
        left = 0
    elif (error_x > 0) and (error_x1 < 0):
        right = 0
        left = 0
    elif error_x == 0 :
        right = 0
        left = 0
    elif (error_x < 0) and (error_x1 <= 0) :
        right = 0
        left = kec
    elif (error_x < 0) and (error_x1 > 0) :
        right = 0
        left = 0

    if (error_y > 0) and (error_y1 >= 0):
        forward = kec
        backward = 0
    elif (error_y > 0) and (error_y1 < 0):
        forward = 0
        backward = 0
    elif error_y == 0 :
        forward = 0
        backward = 0
    elif (error_y < 0) and (error_y1 <= 0) :
        forward = 0
        backward = kec
    elif (error_y < 0) and (error_y1 > 0) :
        forward = 0
        backward = 0
    # print("command : Forward=" + str(forward) + " , Backward=" + str(backward) + " , right=" + str(forward) + " , left=" + str(left))
    move(drone, forward=forward, backward=backward, right=right, left=left)

    error_x1 = error_x
    error_y1 = error_y
    return first, error_x1, error_y1

